In [1]:
import dysts.flows
import pytorch_lightning as pl
import torch
from dysts.base import DynSys
from numpy.random import rand
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import TensorDataset, random_split, DataLoader

from config import ROOT_DIR
from ecodyna.data import generate_trajectories, build_in_out_pair_dataset
from ecodyna.metrics import RNNForecastMetricLogger
from ecodyna.mutitask_models import MultiTaskRNN
from ecodyna.pl_wrappers import LightningForecaster
from ecodyna.plot import plot_3d_trajectories

# Sets random seed for random, numpy and torch
pl.seed_everything(42, workers=True);

Global seed set to 42


In [2]:
data_parameters = {'trajectory_count': 100, 'trajectory_length': 1000}
in_out_parameters = {'n_in': 10, 'n_out': 5}
common_model_parameters = {'model': 'LSTM', 'n_hidden': 32, 'n_layers': 1, **in_out_parameters}
experiment_parameters = {'max_epochs': 50, 'train_part': 0.75}
dataloader_parameters = {'batch_size': 64, 'num_workers': 8}

train_size = int(experiment_parameters['train_part'] * data_parameters['trajectory_count'])
val_size = data_parameters['trajectory_count'] - train_size

In [3]:
def train_rnn_and_plot_different_forecast_types(Model, model_parameters, attractor_name):
    attractor: DynSys = getattr(dysts.flows, attractor_name)()
    attractor.dt *= 10

    attractor_x0 = attractor.ic.copy()
    space_dim = len(attractor_x0)

    data = generate_trajectories(attractor, **data_parameters, ic_fun=lambda: rand(space_dim) - 0.5 + attractor_x0)
    dataset = TensorDataset(data)

    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_data = data[train_dataset.indices]
    val_data = data[val_dataset.indices]

    chunk_train_dataset = build_in_out_pair_dataset(train_dataset, **in_out_parameters)
    chunk_val_dataset = build_in_out_pair_dataset(val_dataset, **in_out_parameters)

    chunk_train_dl = DataLoader(chunk_train_dataset, **dataloader_parameters, shuffle=True)
    chunk_val_dl = DataLoader(chunk_val_dataset, **dataloader_parameters)

    model = Model(space_dim=space_dim, **model_parameters, **common_model_parameters)
    forecaster = LightningForecaster(model=model)

    wandb_logger = WandbLogger(
        save_dir=f'{ROOT_DIR}/results',
        project='notebook-lstm-forecast-types-chunkyness-evaluation-new',
        name=f'{model.name()}_{attractor_name}_{model_parameters["forecast_type"]}'
    )

    wandb_logger.experiment.config.update({
        'forecaster': {'name': model.name(), **model_parameters, **common_model_parameters},
        'data': {'attractor': attractor_name, **data_parameters},
        'dataloader': dataloader_parameters,
        'experiment': experiment_parameters
    })

    trainer = pl.Trainer(max_epochs=experiment_parameters['max_epochs'], callbacks=[EarlyStopping('val_loss', patience=5), RNNForecastMetricLogger(train_dataset, val_dataset)], logger=wandb_logger)

    trainer.fit(forecaster, train_dataloaders=chunk_train_dl, val_dataloaders=chunk_val_dl)

    train_dataloader = DataLoader(train_dataset, **dataloader_parameters)
    val_dataloader = DataLoader(val_dataset, **dataloader_parameters)

    train_preds_chunks = torch.concat(trainer.predict(forecaster, dataloaders=train_dataloader))
    val_preds_chunks = torch.concat(trainer.predict(forecaster, dataloaders=val_dataloader))

    if model_parameters['forecast_type'] == 'recurrent':
        forecaster.prediction_func_name = 'forecast_recurrently_one'
        train_preds_recurrently_one = torch.concat(trainer.predict(forecaster, dataloaders=train_dataloader))
        val_preds_recurrently_one = torch.concat(trainer.predict(forecaster, dataloaders=val_dataloader))

        plot_3d_trajectories(
            labels=['ground truth (train)', 'prediction (chunk by chunk)', 'prediction (recurrently 1 by 1)'],
            tensors=[train_data, train_preds_chunks, train_preds_recurrently_one],
            n_plots=16
        )
        plot_3d_trajectories(
            labels=['ground truth (validation)', 'prediction (chunk by chunk)', 'prediction (recurrently 1 by 1)'],
            tensors=[val_data, val_preds_chunks, val_preds_recurrently_one],
            n_plots=16
        )
    elif model_parameters['forecast_type'] == 'n_out':
        forecaster.prediction_func_name = 'forecast_recurrently_n_out_first'
        train_preds_n_out_first = torch.concat(trainer.predict(forecaster, dataloaders=train_dataloader))
        val_preds_n_out_first = torch.concat(trainer.predict(forecaster, dataloaders=val_dataloader))

        plot_3d_trajectories(
            labels=['ground truth (train)', 'prediction (chunk by chunk)', 'prediction (n_out first)'],
            tensors=[train_data, train_preds_chunks, train_preds_n_out_first],
            n_plots=16
        )
        plot_3d_trajectories(
            labels=['ground truth (validation)', 'prediction (chunk by chunk)', 'prediction (n_out first)'],
            tensors=[val_data, val_preds_chunks, val_preds_n_out_first],
            n_plots=16
        )

    wandb_logger.experiment.finish(quiet=True)

In [4]:
train_rnn_and_plot_different_forecast_types(Model=MultiTaskRNN, model_parameters={'forecast_type': 'recurrent'}, attractor_name='Lorenz')

wandb: Currently logged in as: streitlua (use `wandb login --relogin` to force relogin)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type         | Params
---------------------------------------
0 | model | MultiTaskRNN | 11.7 K
---------------------------------------
11.7 K    Trainable params
0         Non-trainable params
11.7 K    Total params
0.047     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

...

Predicting: 1071it [00:00, ?it/s]

Exception ignored in: <function _releaseLock at 0x7fe5fce37550>
Traceback (most recent call last):
  File "/home/lstreit/anaconda3/envs/chaos/lib/python3.8/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
Exception ignored in: <function _releaseLock at 0x7fe5fce37550>
Traceback (most recent call last):
  File "/home/lstreit/anaconda3/envs/chaos/lib/python3.8/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
Exception ignored in: <function _after_at_fork_child_reinit_locks at 0x7fe5fce37700>
Traceback (most recent call last):
  File "/home/lstreit/anaconda3/envs/chaos/lib/python3.8/logging/__init__.py", line 257, in _after_at_fork_child_reinit_locks
      File "/home/lstreit/anaconda3/envs/chaos/lib/python3.8/_weakrefset.py", line 59, in __iter__
    Exception ignored in: Process wandb_internal:
Traceback (most recent call last):
  File "/home/lstreit/anaconda3/envs/chaos/lib/python3.8/multiprocessing/p

TypeError: concat(): argument 'tensors' (position 1) must be tuple of Tensors, not NoneType

Error in callback <function _WandbInit._pause_backend at 0x7fe5a57ff5e0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [ ]:
train_rnn_and_plot_different_forecast_types(Model=MultiTaskRNN, model_parameters={'forecast_type': 'n_out'}, attractor_name='Lorenz')